![Logo de AA1](logo_AA1_texto_small.png) 
# Sesión 5 - Preprocesado de datos numéricos
Cuando trabajamos con atributos numéricos a veces es necesario realizar algún preprocesado de los mismos para evitar efectos indeseados. Concretamente vamos a abordar los temas:
- escalado de atributos
- normalización de ejemplos
- tratamiento de valores desconocidos

## 5.1 Escalado de atributos
Si los atributos contienen valores con órdenes de magnitud muy diferentes es posible que algunos algoritmos, sobre todo los que realizan cálculo de distancias entre ejemplos, se vean afectados por este hecho. Veámoslo con un ejemplo.

Vamos a cargar los datos contenidos en la pestaña 'Datos' de la hoja de cálculo **ejemplo.xlsx**:

In [1]:
# se importan las librerías
import pandas as pd
from sklearn import preprocessing, impute

df = pd.read_excel('ejemplo.xlsx', sheet_name='Datos')
display(df)

,Altura (cm),Peso (Kg),Salario (euros)
0,165.0,69.3,2645
1,171.4,72.0,1256
2,173.7,72.8,1657
3,219.0,124.0,1723


Vemos que la hoja de cálculo contiene únicamente 4 ejemplos que representan datos de personas, concretamente la altura (en centímetros), el peso (en kilogramos) y el salario mensual (en euros). A simple vista, ya podemos observar que el orden de magnitud es diferente entre los atributos.

Vamos ahora a calcular la distancia euclidea que hay entre los 4 ejemplos. Para ello utilizamos la función `pdist()` que calcula la distacia de todos los ejemplos entre ellos y la función `squareform()` que coloca esas distancias en forma de matriz simétrica donde se identifica de manera sencilla a qué par de ejemplos corresponde cada distancia. Ambas funciones forman parte de la librería `scipy`: https://scipy.org 

In [2]:
from scipy.spatial.distance import pdist, squareform

distances = pdist(df.values, metric='euclidean')
dist_matrix = squareform(distances)
display(dist_matrix)

array([[   0.        , 1389.0173685 ,  988.04450305,  925.19840575],
       [1389.0173685 ,    0.        ,  401.00739395,  472.29096964],
       [ 988.04450305,  401.00739395,    0.        ,   95.02383911],
       [ 925.19840575,  472.29096964,   95.02383911,    0.        ]])

Cada valor de esta matriz presenta la distancia euclídea entre dos ejemplos de tal forma que se corresponden con el orden que presentan los ejemplos en el `DataFrame` original. Así, si nos colocamos en la fila 0 y en la columna 2 vemos un 988.0 que es la distancia entre los ejemplos 0 y 2 del `DataFrame`.

Podemos ver que la menor distancia (95.0) se da entre los ejemplos 2 y 3 y que la mayor distancia (1389.0) se da entre los ejemplos 0 y 1.

En cierto modo, viendo los ejemplos implicados, los resultados pueden chocarnos un poco puesto que la menor distancia se está dando entre dos personas que tienen altura y peso muy diferentes aunque con salarios similares.

Esto se debe a que los valores que se almacenan en el atributo salario son mucho mayores que los que se almacenan en los otros dos atributos y esto provoca que la distancia euclídea se vea dominada por este atributo quitando importancia a los otros dos. Por esta razón, los dos ejemplos más cercanos son los que tienen el salario más similar y los más lejanos los que tienen mayor diferencia en salario.

Para suavizar este problema podemos optar por varias vías de entre las que destacaremos 2 de ellas:
- escalar los atributos entre unos valores máximos y mínimos (`MinMaxScaler()`)
- escalar los atributos para que compartan media y desviación (`StandardScaler()`)

Antes de nada veamos ciertas estadísticas de los atributos del conjunto:

In [3]:
print("\n#### Muestra estadísticas ####")
display(df.describe())



#### Muestra estadísticas ####


,Altura (cm),Peso (Kg),Salario (euros)
count,4.000000,4.000000,4.00000
mean,182.275000,84.525000,1820.25000
std,24.758483,26.359233,587.28152
min,165.000000,69.300000,1256.00000
25%,169.800000,71.325000,1556.75000
50%,172.550000,72.400000,1690.00000
75%,185.025000,85.600000,1953.50000
max,219.000000,124.000000,2645.00000


Vemos que las estadísticas de cada atributo son completamente diferentes.

### 5.1.1 Escalar los atributos entre unos valores máximos y mínimos (`MinMaxScaler()`)

Vamos a aplicar ahora un escalado acotando los valores entre 0 y 1, que son los valores entre los que la función `MinMaxScaler()` escala por defecto: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html 

Para realizar el escalado se aplica la fómula

$$
\hat{v}_i = \frac{v_i - min(v)}{max(v) - min(v)}
$$

donde a cada valor $v_i$ del atributo $v$ se le resta el mínimo de los valores de ese atributo y posteriormente se divide entre la diferencia de sus valores máximo y mínimo.


In [5]:
df_min_max = pd.read_excel('ejemplo.xlsx', sheet_name='Datos')

# se crea un objeto MinMaxScaler con parámetros por defecto
scaler = preprocessing.MinMaxScaler()

# se escalan los atributos que seleccionemos en dos pasos:
# 1. fit: se recorre cada columna calculando el máximo y el mínimo
# 2. transform: se escala cada valor aplicando la fórmula
df_min_max[df_min_max.columns] = scaler.fit_transform(df_min_max[df_min_max.columns])
# se podría haber hecho en dos pasos utilizando primero el método fit() y posteriormente el transform()
# ya veremos esto en sesiones futuras

print("\n#### Muestra el dataframe tras escalar ####")
display(df_min_max)

print("\n#### Muestra estadísticas tras escalar ####")
display(df_min_max.describe())


#### Muestra el dataframe tras escalar ####


,Altura (cm),Peso (Kg),Salario (euros)
0,0.000000,0.000000,1.000000
1,0.118519,0.049360,0.000000
2,0.161111,0.063985,0.288697
3,1.000000,1.000000,0.336213



#### Muestra estadísticas tras escalar ####


,Altura (cm),Peso (Kg),Salario (euros)
count,4.000000,4.000000,4.000000
mean,0.319907,0.278336,0.406228
std,0.458490,0.481887,0.422809
min,0.000000,0.000000,0.000000
25%,0.088889,0.037020,0.216523
50%,0.139815,0.056673,0.312455
75%,0.370833,0.297989,0.502160
max,1.000000,1.000000,1.000000


Tras escalar todos los valores, vemos que están contenidos en el intervalo [0,1]. Si observamos sus estadísticas podemos observar que ahora el orden de magnitud de las medias es similar.

Si calculamos las distancias entre los ejemplos tras la transformación obtendremos lo siguiente:

In [6]:
distances = pdist(df_min_max.values, metric='euclidean')
dist_matrix = squareform(distances)
display(dist_matrix)

array([[0.        , 1.00820785, 0.73212227, 1.56224615],
       [1.00820785, 0.        , 0.29218817, 1.33931512],
       [0.73212227, 0.29218817, 0.        , 1.25782182],
       [1.56224615, 1.33931512, 1.25782182, 0.        ]])

Ahora la mayor distancia (1.56) se presenta entre las personas 0 y 3 (que son las más distantes en peso y altura) y la menor (0.29) entre las personas 1 y 2 (que son las más parecidas en peso y altura). 

Se han llevado todos los atributos al mismo orden de magnitud y ahora el peso de los atributos está más equilibrado.

### 5.1.2 Escalar los atributos para que compartan media y desviación (`StandardScaler()`)

Si observamos nuevamente cómo quedan los ejemplos tras la transformación con `MinMaxScaler()`:

In [7]:
print("\n#### Muestra el dataframe tras escalar ####")
display(df_min_max)


#### Muestra el dataframe tras escalar ####


,Altura (cm),Peso (Kg),Salario (euros)
0,0.000000,0.000000,1.000000
1,0.118519,0.049360,0.000000
2,0.161111,0.063985,0.288697
3,1.000000,1.000000,0.336213


podemos apreciar que como tenemos una persona que es muy diferente a las demás en altura y peso, entonces la mayoría de los valores en esos atributos están muy cercanos un extremos (el 0) mientras que la persona fuera de lo usual se encuentra en el otro extremo.

Para no ser tan rígidos a la hora de escalar y permitir que los valores extraños puedan desviarse un poco de los valores centrales, lo que vamos a hacer es escalar utilizando la estandarización

$$
\hat{v}_i = \frac{v_i - \bar{v}}{\sigma}
$$

donde a cada valor se le restará la media de valores del atributo y se dividirá entre la desviación. Para ello vamos a utilizar la función `StandardScaler()`: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html 

In [8]:
df_stand = pd.read_excel('ejemplo.xlsx', sheet_name='Datos')

# se crea un objeto StandardScaler con parámetros por defecto
scaler = preprocessing.StandardScaler()

# se escalan los atributos que seleccionemos:
df_stand[df_stand.columns] = scaler.fit_transform(df_stand[df_stand.columns])

print("\n#### Muestra el dataframe tras escalar ####")
display(df_stand)

print("\n#### Muestra estadísticas tras escalar ####")
display(df_stand.describe())


#### Muestra el dataframe tras escalar ####


,Altura (cm),Peso (Kg),Salario (euros)
0,-0.805681,-0.666951,1.621606
1,-0.507195,-0.548674,-1.109416
2,-0.399926,-0.513629,-0.320979
3,1.712802,1.729254,-0.191211



#### Muestra estadísticas tras escalar ####


,Altura (cm),Peso (Kg),Salario (euros)
count,4.000000e+00,4.000000e+00,4.000000
mean,1.054712e-15,-2.775558e-16,0.000000
std,1.154701e+00,1.154701e+00,1.154701
min,-8.056815e-01,-6.669510e-01,-1.109416
25%,-5.818163e-01,-5.782432e-01,-0.518088
50%,-4.535602e-01,-5.311514e-01,-0.256095
75%,1.282561e-01,4.709178e-02,0.261993
max,1.712802e+00,1.729254e+00,1.621606


Vemos en las estadísticas que en todos los atributos la media es aproximadamente 0 y la desviación aproximadamente 1. 

Si tuviésemos más ejemplos la media acabaría siendo 0 y la desviación 1. Vemos que la última persona sigue teniendo valores en peso y altura diferentes al resto (esto es inevitable ya que es diferente), pero podemos apreciar también que en peso y altura aumentan las diferencias entre los otros 3 ejemplos.

Vemos que, al igual que sucedía con el escalado anterior, los ejemplos 1 y 2 son los más cercanos mientras que el 0 y el 3 son los más lejanos. 

In [9]:
distances = pdist(df_stand.values, metric='euclidean')
dist_matrix = squareform(distances)
display(dist_matrix)

array([[0.        , 2.74983051, 1.99042228, 3.92056897],
       [2.74983051, 0.        , 0.79647273, 3.3106556 ],
       [1.99042228, 0.79647273, 0.        , 3.08398783],
       [3.92056897, 3.3106556 , 3.08398783, 0.        ]])

¿Es mejor utilizar `StandardScaler()` o `MinMaxScaler()`? Dependerá de los datos a los que nos enfrentemos, pero sí es cierto que **es más habitual aplicar la estandarización**. 

Cuando algún ejemplo presenta valores raros, como por ejemplo una altura fuera de lo común, eso se considera un **outlier** y **NO** es recomendable utilizar `MinMaxScaler()`. En la asignatura *Aprendizaje Automático II* se os hablará de los *outliers* y cómo tratarlos.

## 5.2 Normalización de ejemplos

La normalización es la transformación individual de cada ejemplo para que tenga norma uno.

Lo que hemos visto hasta ahora fue realizar el escalado en base a los valores de la columna; con la normalización vamos a atender únicamente a los valores de cada ejemplo para proceder a la normalización.

Un ejemplo se normaliza dividiendo por la norma cada uno de sus valores:
$$
\hat{x}_i = \frac{x_i}{||x||}
$$
Más info: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer 

La normalización cambia totalmente el significado de los datos porque las distribuciones de los valores de los atributos resultantes quedan totalmente cambiadas. Por lo tanto, un escenario en el que puede ser útil la normalización es cuando se considere más relevante la relación entre los atributos de un mismo ejemplo que con respecto a los atributos del resto de ejemplos. Esto sucede por ejemplo cuando se trabaja con textos (algo que se escapa a los objetivos de esta asignatura pero que se verá en cursos superiores). 

In [10]:
df_norm = pd.read_excel('ejemplo.xlsx', sheet_name='Datos')

print("\n#### Antes de normalizar ####")
display(df_norm)

# se crea un objeto Normalizer: con el parámetro 'norm' podemos elegir la norma deseada
normalizer = preprocessing.Normalizer()

# se normaliza. Hay que destacar que en este caso so es necesario hacer 'fit' puesto que no se necesita
# consultar el resto de ejemplos para poder llevar a cabo la normalización 
df_norm[df_norm.columns] = normalizer.transform(df_norm[df_norm.columns])

print("\n#### Después de normalizar ####")
display(df_norm)

print("\n#### Muestra estadísticas tras escalar ####")
display(df_norm.describe())


#### Antes de normalizar ####


,Altura (cm),Peso (Kg),Salario (euros)
0,165.0,69.3,2645
1,171.4,72.0,1256
2,173.7,72.8,1657
3,219.0,124.0,1723



#### Después de normalizar ####


/home/daniel/.local/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Normalizer was fitted without feature names
  warnings.warn(


,Altura (cm),Peso (Kg),Salario (euros)
0,0.062240,0.026141,0.997719
1,0.134994,0.056707,0.989222
2,0.104157,0.043654,0.993602
3,0.125769,0.071212,0.989500



#### Muestra estadísticas tras escalar ####


,Altura (cm),Peso (Kg),Salario (euros)
count,4.000000,4.000000,4.000000
mean,0.106790,0.049428,0.992511
std,0.032390,0.019176,0.004008
min,0.062240,0.026141,0.989222
25%,0.093678,0.039275,0.989431
50%,0.114963,0.050180,0.991551
75%,0.128076,0.060333,0.994631
max,0.134994,0.071212,0.997719


Si comparamos el `DataFrame()` antes y después de normalizar, podemos apreciar que el ejemplo 3, que representa a la persona más alta, las la normalización tiene una altura de 0.12, que ya no es la mayor. A esto nos referíamos antes, la relación entre los valores de un atributo se ha perdido, con lo que estamos casi ante un nuevo conjunto de datos.

El `StandardScaler()` y otros métodos que trabajan en función de los atributos son utilizados en caso de que la información significativa se encuentre en la relación entre los valores de los atributos entre diferentes ejemplos, mientras que el `Normalizer()` y otros métodos que trabajan en función de las muestras se utilizan en caso de que la información significativa se encuentre en la relación entre los valores de atributos dentro de cada ejemplo.


## 5.3 Tratamiento de valores desconocidos

Ya vimos en una sesión anterior que, a veces, el conjunto con el que estamos trabajando no cuenta con todos los datos, sino que falta alguno de ellos. Esos datos que faltan se conocen como valores desconocidos (missing values).

Algunos algoritmos son capaces de trabajar aunque falte algún valor en los datos, perola mayoría de los algoritmnos necesitan que estén todos los datos disponibles. Por esta razón se suele evitar tener valores desconocidos en el conjunto de datos y, para ello, podemos optar por dos vías:
- eliminar los ejemplos que contienen valores desconocidos
- asignar valores cuando no se conocen

### 5.3.1 Eliminar los ejemplos que contienen valores desconocidos

Es la opción más sencilla y es la más recomendable si tenemos ejemplos de sobra (lo cual no suele ser muy habitual).

Vamos a ver cómo se hace utilizando los datos contenidos en la pestaña 'Missing' de la hoja de cálculo **ejemplo.xlsx**:


In [11]:
df_drop = pd.read_excel('ejemplo.xlsx', sheet_name='Missing', na_values='?')

print("\n#### Antes de eliminar los ejemplos ####")
display(df_drop)
print(df_drop.shape)

# se utiliza dropna() para eliminar ejemplo con missing values
df_drop = df_drop.dropna()

print("\n#### Después de eliminar los ejemplos ####")
display(df_drop)
print(df_drop.shape)



#### Antes de eliminar los ejemplos ####


,Altura (cm),Peso (Kg),Salario (euros al día)
0,165.0,69.3,125.95
1,171.4,72.0,59.81
2,173.7,72.8,78.90
3,219.0,124.0,82.05
4,183.0,95.0,44.14
5,178.0,NaN,116.71
6,191.0,84.0,NaN
7,163.0,68.0,40.24
8,NaN,73.0,63.05
9,194.0,101.0,67.76


(11, 3)

#### Después de eliminar los ejemplos ####


,Altura (cm),Peso (Kg),Salario (euros al día)
0,165.0,69.3,125.95
1,171.4,72.0,59.81
2,173.7,72.8,78.90
3,219.0,124.0,82.05
4,183.0,95.0,44.14
7,163.0,68.0,40.24
9,194.0,101.0,67.76


(7, 3)


Ha sido muy sencillo, lo único que hay que hacer es utilizar el método `dropna()` y ya elimina todos los ejemplos que tengan al menos un valor desconocido. 

En nuestro ejemplo hemos pasado de tener 11 ejemplos a tener 7, lo que supone una reducción importante del número de ejemplos y no suele se asumible.

### 5.3.2 Asignar valores cuando no se conocen

Como es habitual que haya escasez de datos, lo normal es tratar de asignar valores adecuados cuando nos encontramos con algún *missing*. En una mala traducción del inglés esta acción se conoce como "imputar" valores.

Cuando tenemos un ejemplo al que le falta el valor de un atributo, se le suele asignar, por ejemplo, el valor que resulta si se calcula la media de los valores de todos los ejemplos en ese atributo. Pero, ¿por qué la media?. No tiene por qué ser la media, también se le podría asignar la mediana, el valor más frecuente (moda) o un valor constante que sepamos que es coherente.

Esto podemos hacerlo utilizando la clase `SimpleImputer()`, que tiene un parámetro (`strategy`) que nos permite elegir la estrategia que queremos utilizar para la asignación de datos:

Más info: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer

In [12]:
df_simple_imp = pd.read_excel('ejemplo.xlsx', sheet_name='Missing', na_values='?')

print("\n#### Antes de asignar valores a los missing con SimpleImputer ####")
display(df_simple_imp)

# se crea un objeto de SimpleImputer con la media como estrategia
imputer_media = impute.SimpleImputer(strategy='mean')

# se realiza el calculo de las medias y se asigna a los missing
df_simple_imp[df_simple_imp.columns] = imputer_media.fit_transform(df_simple_imp[df_simple_imp.columns])

print("\n#### Después de asignar valores a los missing con SimpleImputer ####")
display(df_simple_imp)


#### Antes de asignar valores a los missing con SimpleImputer ####


,Altura (cm),Peso (Kg),Salario (euros al día)
0,165.0,69.3,125.95
1,171.4,72.0,59.81
2,173.7,72.8,78.90
3,219.0,124.0,82.05
4,183.0,95.0,44.14
5,178.0,NaN,116.71
6,191.0,84.0,NaN
7,163.0,68.0,40.24
8,NaN,73.0,63.05
9,194.0,101.0,67.76



#### Después de asignar valores a los missing con SimpleImputer ####


,Altura (cm),Peso (Kg),Salario (euros al día)
0,165.00,69.300000,125.950
1,171.40,72.000000,59.810
2,173.70,72.800000,78.900
3,219.00,124.000000,82.050
4,183.00,95.000000,44.140
5,178.00,84.344444,116.710
6,191.00,84.000000,78.628
7,163.00,68.000000,40.240
8,181.91,73.000000,63.050
9,194.00,101.000000,67.760


En este caso conservamos todos los ejemplos y los `NaN` han sido sustituidos por el valor medio de cada atributo.

Pero a veces, podríamos realizar asignaciones un poco más inteligentes utilizando el resto de información que contiene el conjunto. Para asignar un valor desconocido podríamos buscar ejemplo que se parezcan y asignarle la media únicamente de los ejemplos que más se parezcan. Eso es lo que hace `KNNImputer()`:

Más info: https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer

In [13]:
df_knn_imp = pd.read_excel('ejemplo.xlsx', sheet_name='Missing', na_values='?')

print("\n#### Antes de asignar valores a los missing con KNNImputer ####")
display(df_knn_imp)

# se crea un objeto de KNNImputer indicando que se utilizan los 2 ejemplos más cercanos
imputer_knn = impute.KNNImputer(n_neighbors=2)

# se realiza el calculo de las medias y se asigna a los missing
df_knn_imp[df_knn_imp.columns] = imputer_knn.fit_transform(df_knn_imp[df_knn_imp.columns])

print("\n#### Después de asignar valores a los missing con KNNImputer ####")
display(df_knn_imp)


#### Antes de asignar valores a los missing con KNNImputer ####


,Altura (cm),Peso (Kg),Salario (euros al día)
0,165.0,69.3,125.95
1,171.4,72.0,59.81
2,173.7,72.8,78.90
3,219.0,124.0,82.05
4,183.0,95.0,44.14
5,178.0,NaN,116.71
6,191.0,84.0,NaN
7,163.0,68.0,40.24
8,NaN,73.0,63.05
9,194.0,101.0,67.76



#### Después de asignar valores a los missing con KNNImputer ####


,Altura (cm),Peso (Kg),Salario (euros al día)
0,165.0,69.30,125.950
1,171.4,72.00,59.810
2,173.7,72.80,78.900
3,219.0,124.00,82.050
4,183.0,95.00,44.140
5,178.0,76.65,116.710
6,191.0,84.00,75.905
7,163.0,68.00,40.240
8,181.2,73.00,63.050
9,194.0,101.00,67.760


En general, `KNNImputer()` suele obtener mejores resultados aunque aquí quizá no se aprecie al ser algo escaso el número de ejemplos disponibles. 

**OJO:** Hay que tener cuidado cuando se utilice `KNNImputer()` ya que como estamos calculando distancias. Si hay un atributo dominante, entonces puede que no estemos seleccionando de manera adecuada a los ejemplos más cercanos y asignaremos un valor incorrecto. 

Esto se debe al problema que veíamos anteriormente de los órdenes de magnitud diferentes entre los atributos y que lo solucionábamos con un escalado. En este ejemplo, no tenemos este efecto ya que el salario lo hemos indicado en "euros al día" (dividiendo entre 21 el mensual) y así que son órdenes de magnitud similares, pero si lo hubiésemos dejado en "euros al mes" entonces obtendríamos resultados incorrectos.

## Ejercicios

Haz un programa que cargue el fichero **mammographic_masses.data** (es un archivo de texto) y realice lo siguiente:

1. Para realizar la carga debes tener en cuenta si tiene o no valores desconocidos y si no tiene cabecera debes asignar nombres a las columnas mediante el parámetro 'names'
2. Haz un escalado en el rango [0,1] y compara los datos antes y despues del escalado. Aplica el escalado a todas las columnas menos a la clase
3. Vuelve a cargar el fichero y repite el apartado 2 realizando una estandarización
4. Carga de nuevo el fichero, elimina los ejemplos con valores desconocidos y compara el conjunto antes y después de la eliminación.
5. Repite el apartado 2 pero en lugar de eliminar los desconocidos trata de asignarles valores.

Estos ejercicios no es necesario entregarlos.